# Categorical EDA (Karim)

Этот ноутбук сделан под задачу Data Fusion Contest.

Что делает:
- быстро загружает `train_labels` из `.parquet` или `.csv`;
- батчами читает `train_part_*.parquet` и берет только размеченные `event_id`;
- считает влияние категориальных фич на `target`;
- строит графики и сохраняет CSV-результаты.


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import chi2_contingency, kruskal
from sklearn.feature_selection import mutual_info_classif

sns.set_theme(style='whitegrid')
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


In [ ]:
# Автоопределение корня проекта
if (Path.cwd() / 'src' / 'data').exists():
    REPO_ROOT = Path.cwd()
elif (Path.cwd().parent / 'src' / 'data').exists():
    REPO_ROOT = Path.cwd().parent
else:
    raise FileNotFoundError('Не нашел папку src/data. Открой ноутбук из корня репозитория или notebooks/.')

DATA_DIR = REPO_ROOT / 'src' / 'data'
OUT_DIR = REPO_ROOT / 'outputs' / 'categorical_analysis'
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('REPO_ROOT =', REPO_ROOT)
print('DATA_DIR =', DATA_DIR)
print('OUT_DIR  =', OUT_DIR)


In [ ]:
# Поиск train labels (parquet/csv)
label_candidates = sorted(DATA_DIR.glob('*label*.parquet')) + sorted(DATA_DIR.glob('*label*.csv'))     + sorted(DATA_DIR.glob('*target*.parquet')) + sorted(DATA_DIR.glob('*target*.csv'))

if not label_candidates:
    raise FileNotFoundError(
        'Не найден labels файл. Положи его в src/data, например train_labels.parquet или train_labels.csv'
    )

LABELS_PATH = label_candidates[0]
TRAIN_FILES = sorted(DATA_DIR.glob('train_part_*.parquet'))

if not TRAIN_FILES:
    raise FileNotFoundError('Не найдены train_part_*.parquet в src/data')

print('Labels file:', LABELS_PATH.name)
print('Train parts:')
for f in TRAIN_FILES:
    print(' -', f.name)


In [ ]:
def load_labels(labels_path: Path) -> pd.DataFrame:
    if labels_path.suffix.lower() == '.parquet':
        labels = pd.read_parquet(labels_path)
    else:
        labels = pd.read_csv(labels_path)

    cols_lower = {c.lower(): c for c in labels.columns}

    event_col = None
    for c in ['event_id', 'eventid', 'event-id']:
        if c in cols_lower:
            event_col = cols_lower[c]
            break

    target_col = None
    for c in ['target', 'label', 'y', 'class']:
        if c in cols_lower:
            target_col = cols_lower[c]
            break

    if event_col is None or target_col is None:
        raise ValueError(
            f'Не нашел event_id/target в labels. Колонки: {list(labels.columns)}'
        )

    out = labels[[event_col, target_col]].rename(columns={event_col: 'event_id', target_col: 'target'}).copy()
    out['event_id'] = pd.to_numeric(out['event_id'], errors='coerce')
    out['target'] = pd.to_numeric(out['target'], errors='coerce')
    out = out.dropna(subset=['event_id', 'target'])

    out['event_id'] = out['event_id'].astype('int64')
    out['target'] = out['target'].astype('int8')
    out = out.drop_duplicates('event_id', keep='last').reset_index(drop=True)
    return out

labels = load_labels(LABELS_PATH)
print('Labels shape:', labels.shape)
print('Target distribution:')
print(labels['target'].value_counts(dropna=False))
print('Positive rate:', labels['target'].mean().round(6))


In [ ]:
# Категориальные колонки из описания + проверка наличия
CATEGORICAL_COLUMNS = [
    'event_type_nm',
    'event_desc',
    'channel_indicator_type',
    'channel_indicator_sub_type',
    'currency_iso_cd',
    'mcc_code',
    'pos_cd',
    'accept_language',
    'browser_language',
    'timezone',
    'operating_system_type',
    'device_system_version',
    'screen_size',
    'developer_tools',
    'phone_voip_call_state',
    'web_rdp_connection',
    'compromised',
]

sample_cols = pd.read_parquet(TRAIN_FILES[0], columns=None).columns.tolist()
available_cols = [c for c in CATEGORICAL_COLUMNS if c in sample_cols]
missing_cols = [c for c in CATEGORICAL_COLUMNS if c not in sample_cols]

print('Available categorical columns:', len(available_cols))
print(available_cols)
if missing_cols:
    print('Missing columns:', missing_cols)


In [ ]:
# Режимы загрузки
QUICK_MODE = False          # True -> быстрее для первичного осмотра
MAX_LABEL_EVENTS = 200_000  # используется только если QUICK_MODE=True
BATCH_SIZE = 500_000
MISSING_TOKEN = '__MISSING__'

label_event_ids = labels['event_id'].tolist()
if QUICK_MODE and len(label_event_ids) > MAX_LABEL_EVENTS:
    rng = np.random.default_rng(42)
    label_event_ids = rng.choice(label_event_ids, size=MAX_LABEL_EVENTS, replace=False).tolist()

label_event_ids = set(label_event_ids)
print('Using label events:', len(label_event_ids))


In [ ]:
def collect_labeled_rows(train_files, label_event_ids, feature_columns, batch_size=500_000):
    out = []
    read_cols = ['event_id'] + feature_columns

    for file_path in train_files:
        pf = pq.ParquetFile(file_path)
        for batch in pf.iter_batches(columns=read_cols, batch_size=batch_size):
            chunk = batch.to_pandas()
            mask = chunk['event_id'].isin(label_event_ids)
            if mask.any():
                out.append(chunk.loc[mask, read_cols].copy())

    if not out:
        return pd.DataFrame(columns=read_cols)

    res = pd.concat(out, ignore_index=True)
    res = res.drop_duplicates('event_id', keep='last')
    return res

labeled_features = collect_labeled_rows(
    TRAIN_FILES,
    label_event_ids=label_event_ids,
    feature_columns=available_cols,
    batch_size=BATCH_SIZE,
)

print('Collected labeled feature rows:', labeled_features.shape)


In [ ]:
df = labels.merge(labeled_features, on='event_id', how='inner')
print('Merged shape:', df.shape)
print('Target rate after merge:', df['target'].mean().round(6))

if df.empty:
    raise RuntimeError('После merge данных 0 строк. Проверь соответствие event_id в train_part и labels.')


In [ ]:
def _prep_cat(s: pd.Series) -> pd.Series:
    return s.astype('string').fillna(MISSING_TOKEN)


def cramers_v(cat: pd.Series, y: pd.Series):
    ctab = pd.crosstab(cat, y)
    if ctab.shape[0] < 2 or ctab.shape[1] < 2:
        return np.nan, np.nan, np.nan

    chi2, p, _, _ = chi2_contingency(ctab)
    n = ctab.to_numpy().sum()
    if n <= 1:
        return chi2, p, np.nan

    phi2 = chi2 / n
    r, k = ctab.shape
    phi2_corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    r_corr = r - ((r - 1) ** 2) / (n - 1)
    k_corr = k - ((k - 1) ** 2) / (n - 1)
    denom = min(k_corr - 1, r_corr - 1)
    v = np.sqrt(phi2_corr / denom) if denom > 0 else np.nan
    return chi2, p, v


def cat_mutual_info(cat: pd.Series, y: pd.Series):
    codes, _ = pd.factorize(cat, sort=False)
    return mutual_info_classif(codes.reshape(-1, 1), y.values, discrete_features=True, random_state=42)[0]


def kruskal_for_cat(cat: pd.Series, y: pd.Series):
    groups = [y[cat == c].values for c in cat.unique()]
    groups = [g for g in groups if len(g) >= 2]
    if len(groups) < 2:
        return np.nan, np.nan
    h, p = kruskal(*groups, nan_policy='omit')
    return h, p


In [ ]:
rows = []
for col in available_cols:
    cat = _prep_cat(df[col])
    y = df['target'].astype('int8')

    chi2_stat, chi2_p, c_v = cramers_v(cat, y)
    mi = cat_mutual_info(cat, y)
    kr_h, kr_p = kruskal_for_cat(cat, y)

    rows.append({
        'feature': col,
        'n_unique': int(cat.nunique(dropna=False)),
        'missing_rate': float((cat == MISSING_TOKEN).mean()),
        'mutual_info': float(mi),
        'chi2_stat': float(chi2_stat) if pd.notna(chi2_stat) else np.nan,
        'chi2_p_value': float(chi2_p) if pd.notna(chi2_p) else np.nan,
        'cramers_v': float(c_v) if pd.notna(c_v) else np.nan,
        'kruskal_h': float(kr_h) if pd.notna(kr_h) else np.nan,
        'kruskal_p_value': float(kr_p) if pd.notna(kr_p) else np.nan,
    })

metrics = pd.DataFrame(rows).sort_values(['mutual_info', 'cramers_v'], ascending=False)
metrics


In [ ]:
# Визуализация топ фич по MI
plt.figure(figsize=(10, 6))
plot_df = metrics.head(12).copy()
sns.barplot(data=plot_df, y='feature', x='mutual_info', color='#4C78A8')
plt.title('Top categorical features by mutual information')
plt.xlabel('Mutual information')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

# Визуализация топ фич по Cramer's V
plt.figure(figsize=(10, 6))
plot_df = metrics.sort_values('cramers_v', ascending=False).head(12).copy()
sns.barplot(data=plot_df, y='feature', x='cramers_v', color='#F58518')
plt.title("Top categorical features by Cramer's V")
plt.xlabel("Cramer's V")
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


In [ ]:
def summarize_categorical(df, feature, min_count=50, top_n=20):
    cat = _prep_cat(df[feature])
    tmp = pd.DataFrame({'cat': cat, 'target': df['target'].astype('int8')})
    summary = tmp.groupby('cat', dropna=False)['target'].agg(['count', 'mean', 'sum']).rename(
        columns={'mean': 'target_rate', 'sum': 'target_positives'}
    )
    summary['target_negatives'] = summary['count'] - summary['target_positives']
    summary = summary[summary['count'] >= min_count].sort_values('target_rate', ascending=False)
    return summary.head(top_n)


def plot_feature_profile(df, feature, min_count=50, top_n=20):
    summary = summarize_categorical(df, feature, min_count=min_count, top_n=top_n)
    display(summary)

    if summary.empty:
        print(f'No categories with count >= {min_count} for {feature}')
        return

    plt.figure(figsize=(10, 6))
    sns.barplot(x=summary['target_rate'].values, y=summary.index.astype(str), color='#54A24B')
    plt.title(f'{feature}: target_rate for top categories (count >= {min_count})')
    plt.xlabel('Target rate')
    plt.ylabel('Category')
    plt.tight_layout()
    plt.show()


In [ ]:
# Посмотри профили для самых сильных фич
TOP_FEATURES_TO_INSPECT = metrics['feature'].head(5).tolist()
TOP_FEATURES_TO_INSPECT


In [ ]:
for feat in TOP_FEATURES_TO_INSPECT:
    print('
' + '=' * 80)
    print('Feature:', feat)
    plot_feature_profile(df, feat, min_count=30, top_n=20)


In [ ]:
# Сохранение результатов
metrics.to_csv(OUT_DIR / 'categorical_feature_metrics.csv', index=False)

feature_summary_dir = OUT_DIR / 'feature_summaries'
feature_summary_dir.mkdir(parents=True, exist_ok=True)
for feat in available_cols:
    summary = summarize_categorical(df, feat, min_count=1, top_n=1000000)
    summary.to_csv(feature_summary_dir / f'{feat}_summary.csv')

print('Saved:')
print(' -', OUT_DIR / 'categorical_feature_metrics.csv')
print(' -', feature_summary_dir)


## Как интерпретировать

- `mutual_info` и `cramers_v` выше -> признак обычно полезнее.
- Очень высокий `missing_rate` не значит, что фича плохая: иногда сам факт пропуска информативен.
- Смотри не только на статистику, но и на стабильность: категории с очень маленьким `count` часто шумные.
